In [1]:
import os

In [2]:
%pwd

'd:\\Computer_Vision\\pouravi_computer_vision_project\\cotton_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Computer_Vision\\pouravi_computer_vision_project\\cotton_disease_classification'

In [5]:
import tensorflow as tf


In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    testing_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

        
    def get_evaluation_config(self) -> EvaluationConfig:
        self.train = self.config.training
        eval_config = EvaluationConfig(
            path_of_model= Path(self.train.trained_model_path),
            testing_data = Path(os.path.join(self.config.data_ingestion.unzip_dir, "test")),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
config = read_yaml(CONFIG_FILE_PATH)
print(config.training.trained_model_path)

[2024-05-06 20:02:54,763: INFO: common: yaml file: config\config.yaml loaded successfully]
artifacts/training/model.h5


In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _evaluation_generator(self):

        testgenerator_kwargs = dict(rescale = 1./255)
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **testgenerator_kwargs
            )
        self.test_generator = test_datagenerator.flow_from_directory(directory = self.config.testing_data,
                                                 target_size = self.config.params_image_size[:-1],
                                                 batch_size = self.config.params_batch_size,
                                                 class_mode = 'categorical')

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(Path(self.config.path_of_model))
        self._evaluation_generator()
        self.score = self.model.evaluate(self.test_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [17]:
try:
    config = ConfigurationManager()
    eva_config = config.get_evaluation_config()
    evaluation = Evaluation(eva_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-05-07 18:03:38,992: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-07 18:03:38,999: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-07 18:03:39,002: INFO: common: created directory at: artifacts]
Found 18 images belonging to 4 classes.
2/2 [==============================] - 4s 142ms/step - loss: 2.7109 - accuracy: 0.7222
[2024-05-07 18:03:49,191: INFO: common: json file saved at: scores.json]
